In [0]:

## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 200kB/s 
ERROR: tensorflow-model-optimization 0.2.1 requires enum34~=1.1, which is not installed.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2


In [0]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

TensorFlow 2.x selected.
2.1.0
/device:GPU:0
Num GPUs Available:  1


In [0]:
from tensorflow.keras.callbacks import Callback

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
np.random.seed(12227)

In [0]:
n_ep = 200
loss_ = 0.2
bs = 1000

In [0]:
def custom_stopping(value=0.5, verbose=0):
    early = EarlyStoppingByLossVal(monitor='val_loss', value=value, verbose=verbose)
    return early

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_acc', value=0.95, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        # if current is None:
        # warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [0]:
def DataPreparation(data_input_file):
  print('Panwar  et al. 2015 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  # X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  # classes_number = Y.shape[1]
  # Y = np.argmax(Y, axis=1)
  return X,Y,folds

In [0]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [0]:
def zero_padding_MHEALTH(X):
    # Groups the heterogenous sensors for MHEALTH
    idx_modalities = []
    idx_acc = [0, 1, 2, 5, 6, 7, 14, 15, 16]
    idx_gyro = [8, 9, 10, 17, 18, 19]
    idx_mag = [11, 12, 13, 20, 21, 22]
    idx_ele = [3, 4]
    X_acc = X[:, :, :, idx_acc]
    X_gyro = X[:, :, :, idx_gyro]
    X_mag = X[:, :, :, idx_mag]
    X_ele = X[:, :, :, idx_ele]
    X_zeros = np.zeros((X.shape[0], X.shape[1], X.shape[2], 2))  # Vertical Kernel-1 = 2

    X = X_acc
    X = np.concatenate((X, X_zeros), axis=3)
    idx_modalities.append(X.shape[3])

    X = np.concatenate((X, X_gyro), axis=3)
    X = np.concatenate((X, X_zeros),axis=3)
    idx_modalities.append(X.shape[3])

    X = np.concatenate((X, X_mag),axis=3)
    X = np.concatenate((X, X_zeros),axis=3)
    idx_modalities.append(X.shape[3])
    X = np.concatenate((X, X_ele),axis=3)
    return X

In [0]:
def RunPanware(data_input_file):
  X,Y,folds,= DataPreparation(data_input_file)
  # idx = [3, 5, 8, 11, 14, 17, 20] #For MHEALTH
  # idx = [1, 4, 7, 10, 13, 14, 17, 20, 23, 26, 27, 30, 33, 36]#For PAMAP2P
  # idx = [val for val in idx for _ in (0, 1)] #Vertical Kernel-1
  # X = zero_padding_MHEALTH(X)
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [0]:
def build_model_1(row,col,num_classes,inp):

  #Architecture1 from paper
  # activation = 'relu'

  # H = tf.keras.layers.Conv2D(filters=5, kernel_size=(9, 3), padding = 'same')(inp)
  # H = tf.keras.layers.Conv2D(filters=5, kernel_size=(5, 3), padding = 'same')(H)
  # H = tf.keras.layers.MaxPooling2D(pool_size=(2, 1))(H)


  # H = tf.keras.layers.Activation('relu')(H)

  # H = tf.keras.layers.Flatten()(H)
  # H = tf.keras.layers.Dense(num_classes)(H)
  # H = tf.keras.layers.Activation('softmax')(H)

  # model = Model([inp], H)

 
  layers = [
        tf.keras.layers.Conv2D(filters=5,kernel_size=(9, 3), input_shape=(1,row,col)),
        
        tf.keras.layers.Conv2D(5,(5, 3)),
        tf.keras.layers.MaxPool2D((2, 1)),
        tf.keras.layers.Activation('relu'),      

       

       
        tf.keras.layers.Flatten(),
       
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    
  model = tf.keras.Sequential(layers)
  return model

In [0]:
def build_model_2(row,col,num_classes):
  tf.initializers.Orthogonal(gain=1.0, seed=None)
  layers = [
        tf.keras.layers.Conv2D(5,(9, 3),padding='same'),
        tf.keras.layers.Conv2D(5,(7, 3),padding='same'),
        tf.keras.layers.Conv2D(5,(5, 3),padding='same'),
        tf.keras.layers.MaxPool2D((2, 1)),
        tf.keras.layers.Activation('relu'),      

       

       
        tf.keras.layers.Flatten(),
       
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    
  model = tf.keras.Sequential(layers)
  return model

In [0]:
def Train(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _,_, img_rows, img_cols = X.shape
  print(X.shape)
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    model=build_model_1(img_rows,img_cols,n_class,tf.keras.layers.Input((1, img_rows, img_cols)))
    # print(model.summary)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='SGD')
    model.fit(X_train, y[train_idx], batch_size=bs, epochs=n_ep,
                  verbose=0, callbacks=[custom_stopping(value=loss_, verbose=1)],validation_data=(X_train, y[train_idx]))
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [0]:
tf.keras.backend.set_image_data_format('channels_first')

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz
(2555, 1, 250, 23)
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[1]
________________________________________________________________
Accuracy[0.0875] Recall[0.0833] F1[0.0134] at fold[2]
________________________________________________________________
Accuracy[0.0916] Recall[0.0833] F1[0.0140] at fold[3]
________________________________________________________________
Accuracy[0.0865] Recall[0.0833] F1[0.0133] at fold[4]
________________________________________________________________
Accuracy[0.0868] Recall[0.0833] F1[0.0133] at fold[5]
________________________________________________________________
Accuracy[0.0871] Recall[0.0833] F1[0.0134] at fold[6]
________________________________________________________________
Accuracy[0.0909] Recall[0.0833] F1[0.0139] at fold[7]
________________________________________________________________
Accuracy[0.0962] Recall[0.0833] F1[0.0146] at fold[8]
____

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOSO/USCHAD.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOSO/USCHAD.npz
(9824, 1, 500, 6)
Accuracy[0.1008] Recall[0.0833] F1[0.0153] at fold[1]
________________________________________________________________
Accuracy[0.0948] Recall[0.0833] F1[0.0144] at fold[2]
________________________________________________________________
Accuracy[0.0923] Recall[0.0833] F1[0.0141] at fold[3]
________________________________________________________________
Accuracy[0.2059] Recall[0.0833] F1[0.0285] at fold[4]
________________________________________________________________
Accuracy[0.1218] Recall[0.0833] F1[0.0181] at fold[5]
________________________________________________________________
Accuracy[0.1329] Recall[0.0833] F1[0.0196] at fold[6]
________________________________________________________________
Accuracy[0.1528] Recall[0.0833] F1[0.0221] at fold[7]
________________________________________________________________
Accuracy[0.2032] Recall[0.0833] F1[0.0281] at fold[8]
______

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2003: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2004: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz
(3771, 1, 50, 6)
Accuracy[0.0490] Recall[0.0476] F1[0.0044] at fold[1]
________________________________________________________________
Accuracy[0.0538] Recall[0.0476] F1[0.0049] at fold[2]
________________________________________________________________
Accuracy[0.0632] Recall[0.0476] F1[0.0057] at fold[3]
________________________________________________________________
Accuracy[0.0480] Recall[0.0476] F1[0.0044] at fold[4]
________________________________________________________________
Accuracy[0.0524] Recall[0.0476] F1[0.0047] at fold[5]
________________________________________________________________
Accuracy[0.0612] Recall[0.0476] F1[0.0055] at fold[6]
________________________________________________________________
Accuracy[0.0459] Recall[0.0476] F1[0.0042] at fold[7]
________________________________________________________________
Accuracy[0.0446] Recall[0.0476] F1[0.0041] at fold[8]
_

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2003: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2004: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz
(1137, 1, 50, 6)
Accuracy[0.5600] Recall[0.5949] F1[0.5406] at fold[1]
________________________________________________________________
Accuracy[0.7273] Recall[0.7197] F1[0.7180] at fold[2]
________________________________________________________________
Accuracy[0.4868] Recall[0.4882] F1[0.5051] at fold[3]
________________________________________________________________
Accuracy[0.1301] Recall[0.1667] F1[0.0384] at fold[4]
________________________________________________________________
Accuracy[0.3725] Recall[0.4199] F1[0.3988] at fold[5]
________________________________________________________________
Accuracy[0.4943] Recall[0.4833] F1[0.4797] at fold[6]
________________________________________________________________
Accuracy[0.1667] Recall[0.1667] F1[0.0476] at fold[7]
________________________________________________________________
Accuracy[0.3129] Recall[0.3184] F1[0.3127] at fold[8]
_

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz
(1335, 1, 250, 23)
Accuracy[0.0889] Recall[0.0833] F1[0.0136] at fold[1]
________________________________________________________________
Accuracy[0.0889] Recall[0.0833] F1[0.0136] at fold[2]
________________________________________________________________
Accuracy[0.0896] Recall[0.0833] F1[0.0137] at fold[3]
________________________________________________________________
Accuracy[0.0896] Recall[0.0833] F1[0.0137] at fold[4]
________________________________________________________________
Accuracy[0.0896] Recall[0.0833] F1[0.0137] at fold[5]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[6]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[7]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[8]
____

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/FNOW/USCHAD.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/FNOW/USCHAD.npz
(5038, 1, 500, 6)
Accuracy[0.1336] Recall[0.0833] F1[0.0196] at fold[1]
________________________________________________________________
Accuracy[0.1339] Recall[0.0833] F1[0.0197] at fold[2]
________________________________________________________________
Accuracy[0.1347] Recall[0.0833] F1[0.0198] at fold[3]
________________________________________________________________
Accuracy[0.1349] Recall[0.0833] F1[0.0198] at fold[4]
________________________________________________________________
Accuracy[0.1349] Recall[0.0833] F1[0.0198] at fold[5]
________________________________________________________________
Accuracy[0.1349] Recall[0.0833] F1[0.0198] at fold[6]
________________________________________________________________
Accuracy[0.1352] Recall[0.0833] F1[0.0198] at fold[7]
________________________________________________________________
Accuracy[0.1357] Recall[0.0833] F1[0.0199] at fold[8]
______

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD1_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD1_1s.npz
(2048, 1, 50, 6)
Accuracy[0.0516] Recall[0.0476] F1[0.0047] at fold[1]
________________________________________________________________
Accuracy[0.0516] Recall[0.0476] F1[0.0047] at fold[2]
________________________________________________________________
Accuracy[0.0524] Recall[0.0476] F1[0.0047] at fold[3]
________________________________________________________________
Accuracy[0.0529] Recall[0.0476] F1[0.0048] at fold[4]
________________________________________________________________
Accuracy[0.0534] Recall[0.0476] F1[0.0048] at fold[5]
________________________________________________________________
Accuracy[0.0534] Recall[0.0476] F1[0.0048] at fold[6]
________________________________________________________________
Accuracy[0.0545] Recall[0.0476] F1[0.0049] at fold[7]
________________________________________________________________
Accuracy[0.0553] Recall[0.0476] F1[0.0050] at fold[8]
_

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2003: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2004: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD2_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD2_1s.npz
(616, 1, 50, 6)
Accuracy[0.3968] Recall[0.4030] F1[0.4040] at fold[1]
________________________________________________________________
Accuracy[0.5873] Recall[0.5755] F1[0.5863] at fold[2]
________________________________________________________________
Accuracy[0.4603] Recall[0.4651] F1[0.4514] at fold[3]
________________________________________________________________
Accuracy[0.4921] Recall[0.4909] F1[0.4848] at fold[4]
________________________________________________________________
Accuracy[0.5082] Recall[0.5321] F1[0.5228] at fold[5]
________________________________________________________________
Accuracy[0.4754] Recall[0.4904] F1[0.4790] at fold[6]
________________________________________________________________
Accuracy[0.5410] Recall[0.5417] F1[0.5393] at fold[7]
________________________________________________________________
Accuracy[0.4754] Recall[0.4704] F1[0.4605] at fold[8]
__

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOTO/MHEALTH.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOTO/MHEALTH.npz
(2555, 1, 250, 23)
Accuracy[0.0898] Recall[0.0833] F1[0.0137] at fold[1]
________________________________________________________________
Accuracy[0.0895] Recall[0.0833] F1[0.0137] at fold[2]
________________________________________________________________
Accuracy[0.0858] Recall[0.0833] F1[0.0132] at fold[3]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[4]
________________________________________________________________
Accuracy[0.0875] Recall[0.0833] F1[0.0134] at fold[5]
________________________________________________________________
Accuracy[0.0836] Recall[0.0833] F1[0.0129] at fold[6]
________________________________________________________________
Accuracy[0.0885] Recall[0.0833] F1[0.0135] at fold[7]
________________________________________________________________
Accuracy[0.1004] Recall[0.0833] F1[0.0152] at fold[8]
____

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOTO/USCHAD.npz')


Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOTO/USCHAD.npz
(9824, 1, 500, 6)
Accuracy[0.1645] Recall[0.0833] F1[0.0235] at fold[1]
________________________________________________________________
Accuracy[0.1282] Recall[0.0833] F1[0.0189] at fold[2]
________________________________________________________________
Accuracy[0.1295] Recall[0.0833] F1[0.0191] at fold[3]
________________________________________________________________
Accuracy[0.1308] Recall[0.0833] F1[0.0193] at fold[4]
________________________________________________________________
Accuracy[0.1176] Recall[0.0833] F1[0.0175] at fold[5]
________________________________________________________________
Accuracy[0.1250] Recall[0.0833] F1[0.0185] at fold[6]
________________________________________________________________
Accuracy[0.1350] Recall[0.0833] F1[0.0198] at fold[7]
________________________________________________________________
Accuracy[0.1888] Recall[0.0833] F1[0.0265] at fold[8]
______

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD1_1s.npz')


Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD1_1s.npz
(3771, 1, 50, 6)
Accuracy[0.0475] Recall[0.0476] F1[0.0043] at fold[1]
________________________________________________________________
Accuracy[0.0530] Recall[0.0476] F1[0.0048] at fold[2]
________________________________________________________________
Accuracy[0.0546] Recall[0.0476] F1[0.0049] at fold[3]
________________________________________________________________
Accuracy[0.0532] Recall[0.0476] F1[0.0048] at fold[4]
________________________________________________________________
Accuracy[0.0582] Recall[0.0476] F1[0.0052] at fold[5]
________________________________________________________________
Accuracy[0.0551] Recall[0.0476] F1[0.0050] at fold[6]
________________________________________________________________
Accuracy[0.0532] Recall[0.0476] F1[0.0048] at fold[7]
________________________________________________________________
Accuracy[0.0529] Recall[0.0476] F1[0.0048] at fold[8]
_

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2003: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2004: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD2_1s.npz')


Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD2_1s.npz
(1137, 1, 50, 6)
Accuracy[0.5449] Recall[0.5840] F1[0.5478] at fold[1]
________________________________________________________________
Accuracy[0.5781] Recall[0.5502] F1[0.5385] at fold[2]
________________________________________________________________
Accuracy[0.6415] Recall[0.6743] F1[0.6457] at fold[3]
________________________________________________________________
Accuracy[0.6100] Recall[0.6110] F1[0.5960] at fold[4]
________________________________________________________________
Accuracy[0.1504] Recall[0.1667] F1[0.0436] at fold[5]
________________________________________________________________
Accuracy[0.1545] Recall[0.1667] F1[0.0446] at fold[6]
________________________________________________________________
Accuracy[0.5536] Recall[0.5886] F1[0.5561] at fold[7]
________________________________________________________________
Accuracy[0.6300] Recall[0.6214] F1[0.6060] at fold[8]
_

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/SNOW/MHEALTH.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/SNOW/MHEALTH.npz
(2555, 1, 250, 23)
Accuracy[0.0891] Recall[0.0833] F1[0.0136] at fold[1]
________________________________________________________________
Accuracy[0.0898] Recall[0.0833] F1[0.0137] at fold[2]
________________________________________________________________
Accuracy[0.0898] Recall[0.0833] F1[0.0137] at fold[3]
________________________________________________________________
Accuracy[0.0898] Recall[0.0833] F1[0.0137] at fold[4]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[5]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[6]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[7]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[8]
____

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/SNOW/USCHAD.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/SNOW/USCHAD.npz
(9824, 1, 500, 6)
Accuracy[0.1377] Recall[0.0833] F1[0.0202] at fold[1]
________________________________________________________________
Accuracy[0.1377] Recall[0.0833] F1[0.0202] at fold[2]
________________________________________________________________
Accuracy[0.1379] Recall[0.0833] F1[0.0202] at fold[3]
________________________________________________________________
Accuracy[0.1381] Recall[0.0833] F1[0.0202] at fold[4]
________________________________________________________________
Accuracy[0.1386] Recall[0.0833] F1[0.0203] at fold[5]
________________________________________________________________
Accuracy[0.1388] Recall[0.0833] F1[0.0203] at fold[6]
________________________________________________________________
Accuracy[0.1389] Recall[0.0833] F1[0.0203] at fold[7]
________________________________________________________________
Accuracy[0.1389] Recall[0.0833] F1[0.0203] at fold[8]
______

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD1_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD1_1s.npz
(3771, 1, 50, 6)
Accuracy[0.0519] Recall[0.0476] F1[0.0047] at fold[1]
________________________________________________________________
Accuracy[0.0524] Recall[0.0476] F1[0.0047] at fold[2]
________________________________________________________________
Accuracy[0.0525] Recall[0.0476] F1[0.0048] at fold[3]
________________________________________________________________
Accuracy[0.0525] Recall[0.0476] F1[0.0048] at fold[4]
________________________________________________________________
Accuracy[0.0526] Recall[0.0476] F1[0.0048] at fold[5]
________________________________________________________________
Accuracy[0.0531] Recall[0.0476] F1[0.0048] at fold[6]
________________________________________________________________
Accuracy[0.0532] Recall[0.0476] F1[0.0048] at fold[7]
________________________________________________________________
Accuracy[0.0509] Recall[0.0476] F1[0.0046] at fold[8]
_

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2003: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2004: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD2_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD2_1s.npz
(1137, 1, 50, 6)
Accuracy[0.4831] Recall[0.4840] F1[0.4543] at fold[1]
________________________________________________________________
Accuracy[0.5763] Recall[0.5662] F1[0.5718] at fold[2]
________________________________________________________________
Accuracy[0.5603] Recall[0.5230] F1[0.5351] at fold[3]
________________________________________________________________
Accuracy[0.6460] Recall[0.6304] F1[0.6388] at fold[4]
________________________________________________________________
Accuracy[0.5268] Recall[0.5076] F1[0.5148] at fold[5]
________________________________________________________________
Accuracy[0.1518] Recall[0.1667] F1[0.0439] at fold[6]
________________________________________________________________
Accuracy[0.5625] Recall[0.5759] F1[0.5638] at fold[7]
________________________________________________________________
Accuracy[0.5893] Recall[0.6036] F1[0.5699] at fold[8]
_